# Lab 12: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
from localization_extras import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

import nest_asyncio
nest_asyncio.apply()

# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

2022-05-18 14:30:16,540 | INFO     |: Logger demo_notebook.log initialized.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

In [2]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2022-05-18 14:30:20,534 | INFO     |:  | Number of observations per grid cell: 18
2022-05-18 14:30:20,537 | INFO     |:  | Precaching Views...


/Users/trueshot/Documents/git/fast-robots/lab13/ECE4960-sim-release/localization.py:144: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[


2022-05-18 14:30:26,060 | INFO     |:  | Precaching Time: 5.521 secs
2022-05-18 14:30:26,061 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-18 14:30:26,062 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run the Bayes Filter
The cells below utilizes the member functions of class **Localization** (defined in [localization_extras.py](../localization_extras.py)) in each iteration of the Bayes filter algorithm to localize the robot in the grid map. <br>

In [3]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2022-05-18 14:30:28,646 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-18 14:30:28,647 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2022-05-18 14:30:31,560 | INFO     |: Update Step
2022-05-18 14:30:31,565 | INFO     |:      | Update Time: 0.003 secs
2022-05-18 14:30:31,566 | INFO     |: ---------- UPDATE STATS -----------
2022-05-18 14:30:31,571 | INFO     |: GT index      : (6, 4, 9)
2022-05-18 14:30:31,572 | INFO     |: Bel index     : (5, 3, 9) with prob = 0.9986121
2022-05-18 14:30:31,574 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2022-05-18 14:30:31,577 | INFO     |: GT            : (0.000, 0.000, 360.000)
2022-05-18 14:30:31,580 | INFO     |: Belief        : (0.000, -0.305, 10.000)
2022-05-18 14:30:31,581 | INFO     |: POS ERROR     : (-0.000, 0.305, 350.000)
2022-05-18 14:30:31,584 | INFO     |: ---------- UPDATE STATS -----------


In [4]:
np.count_nonzero(loc.bel[loc.bel>0.0001])

1944

In [5]:
np.unique(loc.bel)

array([0.0005144])

In [4]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------
2022-05-18 14:30:37,292 | INFO     |: Prediction Step
2022-05-18 14:30:37,433 | INFO     |:  | Prediction Time: 0.139 secs
2022-05-18 14:30:37,444 | INFO     |: ---------- PREDICTION STATS -----------
2022-05-18 14:30:37,454 | INFO     |: GT index         : (6, 3, 7)
2022-05-18 14:30:37,455 | INFO     |: Prior Bel index  : (5, 5, 8) with prob = 0.0992426
2022-05-18 14:30:37,457 | INFO     |: POS ERROR        : (0.267, -0.389, -12.059)
2022-05-18 14:30:37,460 | INFO     |: ---------- PREDICTION STATS -----------
2022-05-18 14:30:40,436 | INFO     |: Update Step
2022-05-18 14:30:40,440 | INFO     |:      | Update Time: 0.003 secs
2022-05-18 14:30:40,441 | INFO     |: ---------- UPDATE STATS -----------
2022-05-18 14:30:40,449 | INFO     |: GT index      : (6, 3, 7)
2022-05-18 14:30:40,452 | INFO     |: Bel index     : (6, 4, 7) with prob = 1.0
2022-05-18 14:30:40,455 | INFO     |: Bel_bar prob at index = 0.0004770035688837416
2022-05-18 14:30:40,45

KeyboardInterrupt: 

Loading Flatland...
Initializing pygame framework...
